In [1]:
using Pkg
Pkg.activate(".")

  Activating project at `~/local_Documents/CausalInferenceWithR`


In [10]:
Pkg.add(["CSV", "DataFrames", "HTTP", "Statistics"])
using CSV, DataFrames, HTTP, Statistics

   Resolving package versions...
    Updating `~/local_Documents/CausalInferenceWithR/Project.toml`
  [10745b16] + Statistics
  No Changes to `~/local_Documents/CausalInferenceWithR/Manifest.toml`


In [84]:
data02 = CSV.read(HTTP.get("https://raw.githubusercontent.com/mtakahashi123/causality/main/data02.csv").body, DataFrame; missingstring = "NA")

,x1,y3,t1,y0,y1,y0t,y1t
,Int64,Int64,Int64,Int64?,Int64?,Int64,Int64
1,74,76,1,missing,76,68,76
2,82,75,0,75,missing,75,84
3,72,75,1,missing,75,65,75
4,96,84,0,84,missing,84,97
5,83,75,0,75,missing,75,84
6,72,74,1,missing,74,65,74
7,85,76,0,76,missing,76,87
8,87,77,0,77,missing,77,89
9,86,77,0,77,missing,77,87


In [85]:
describe(data02) # 記述統計量の算出

,variable,mean,min,median,max,nmissing,eltype
,Symbol,Float64,Int64,Float64,Int64,Int64,Type
1,x1,81.95,58,83.5,96,0,Int64
2,y3,76.6,61,76.5,87,0,Int64
3,t1,0.3,0,0.0,1,0,Int64
4,y0,77.7857,72,77.0,87,6,"Union{Missing, Int64}"
5,y1,73.8333,61,75.5,80,14,"Union{Missing, Int64}"
6,y0t,73.8,52,75.0,87,0,Int64
7,y1t,83.85,61,84.5,97,0,Int64


- x1~ Normal(80, 10)
- t1 -> 処置割り付けの二値変数
- y0t ~ Normal(80, 9)
- y1t ~ Normal(90, 9)
- y0 x1が80以上のときに観測される変数（期末試験0, Y|T=0）
- y1 x1が80未満のときに観測される変数（期末試験1, Y|T=1)
- y3 y0とy1の観測部分をくっつけた変数

## 処置効果1：個体因果効果
Individual Causal Effect, ICE, もしくはIndividual Treatment Effect, ITE

ICEは単純に，個体ごとの「潜在的結果1-潜在的結果0」と定義される。

In [86]:
# せっかくなのでattach関数を実装する。
function attach(x::DataFrame)
    for i in propertynames(x)
        @eval $(i) = $(x).$(i)
    end
end

attach (generic function with 1 method)

In [87]:
attach(data02)

個体処置効果, ICE

定義はできても，一部に観測できない量があるため，推定できない。

In [88]:
y1t - y0t

20-element Vector{Int64}:
  8
  9
 10
 13
  9
  9
 11
 12
 10
 10
  9
 10
 10
  9
 12
 12
 10
  9
 10
  9

## 処置効果2：平均処置効果

Average Treatment Effect

期待値の加法性から和の期待値は期待値の和に変形できる。

$$
\tau = E[Y_i(1) - Y_i(0)] = E[Y_i(1)] - E[Y_i(0)]
$$

実際には同一個体の$Y_i(1)$と$Y_i(0)$は同時に観測されないので，直接ATEを計算することはできない。

In [89]:
mean(y1t) - mean(y0t)

10.049999999999997

In [90]:
mean(y3) - mean(x1)　# 実際に観測できるデータから計算した事前事後比較

-5.3500000000000085

In [91]:
# ナイーブにペアワイズ除去した場合の，期末試験得点の差
m1 = mean(skipmissing(y1)) 
m0 = mean(skipmissing(y0))
m1 - m0

-3.952380952380963

## 処置効果3：処置群の平均処置効果

Averate Treatment effect on the Treated, ATE

実際に処置を受けた群における効果（政策評価や心理実験などでの関心）

$$
\tau_{ATT} = E[Y_i(1) - Y_i(0)|T_i = 1] = E[Y_i(1)|T_i = 1] - E[Y_i(0)|T_i = 1]
$$

ATEとATTのどちらを使うかは研究テーマによる。手法によってもどちらが推定可能かが変わってくる。

In [92]:
mt1 = mean(y1t[@. t1 == 1])
mt0 = mean(y0t[@. t1 == 1])
mt1 - mt0

9.333333333333329

## 交絡因子

ナイーブな推定量$(y3 - x1)$における推論が「補習ををすると得点が下がる」と結論付けられてしまう。

= 何らかのcofoundingが働いている。

= そもそも補習を受ける人たちは数学が苦手な傾向にあるはずなので，補習授業の効果があったとしても，それを覆い隠すほどに初期位置の数学力が低い可能性がある。

## DAG

## 無作為割付けによる分析の例

In [16]:
Pkg.add(["Random", "Distributions"])
using Random, Distributions

   Resolving package versions...
   Installed FillArrays ──── v0.13.0
   Installed Distributions ─ v0.25.49
    Updating `~/local_Documents/CausalInferenceWithR/Project.toml`
  [31c24e10] + Distributions v0.25.49
  [9a3f8284] + Random
    Updating `~/local_Documents/CausalInferenceWithR/Manifest.toml`
  [d360d2e6] + ChainRulesCore v1.12.0
  [9e997f8a] + ChangesOfVariables v0.1.2
  [b429d917] + DensityInterface v0.4.0
  [31c24e10] + Distributions v0.25.49
  [ffbed154] + DocStringExtensions v0.8.6
  [1a297f60] + FillArrays v0.13.0
  [3587e190] + InverseFunctions v0.1.2
  [92d709cd] + IrrationalConstants v0.1.1
  [692b3bcd] + JLLWrappers v1.4.1
  [2ab3a3ac] + LogExpFunctions v0.3.6
  [90014a1f] + PDMats v0.11.5
  [21216c6a] + Preferences v1.2.3
  [1fd47b50] + QuadGK v2.4.2
  [79098fc4] + Rmath v0.7.0
  [276daf66] + SpecialFunctions v2.1.2
  [82ae8749] + StatsAPI v1.2.1
  [2913bbd2] + StatsBase v0.33.16
  [4c63d2b9] + StatsFuns v0.9.15
  [efe28fd5] + OpenSpecFun_jll v0.5.5+0
  [f50d1b31] +

In [113]:
Random.seed!(1) # 便宜上おなじシードを用いるが，当然Rの結果とは一致しない（乱数生成器も違うし）

TaskLocalRNG()

In [121]:
r0 = rand(Uniform(0, 1), 20)
r1 = @. round(r0; digits = 0)
# r1 = [0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1,] # 検算用
y2 = similar(y1t)
y2[@. r1 == 1] = copy(y1t[@. r1 == 1])
y2[@. r1 == 0] = copy(y0t[@. r1 == 0])

9-element Vector{Int64}:
 68
 75
 75
 70
 87
 75
 52
 72
 80

In [122]:
r1'

1×20 adjoint(::Vector{Int64}) with eltype Int64:
 0  0  1  1  0  1  1  1  1  0  0  0  1  0  1  0  1  1  0  1

In [123]:
mr1 = mean(y2[@. r1 == 1])

85.18181818181819

In [124]:
mr0 = mean(y2[@. r1 == 0])

72.66666666666667

In [125]:
mr1 - mr0 # サンプルサイズが小さすぎるので，結果はかなり不安定。

12.515151515151516

## 2標本$t$検定

無作為割付けを実施したデータで2群間に差についてのt検定を行う。Rコードでは`var.equal = FALSE`で，等分散性の仮定を置かないようにしているので，`HypothesisTests.jl`の`UnequalVarianceTTest()`を使う。

In [119]:
Pkg.add("HypothesisTests")
using HypothesisTests

   Resolving package versions...
  No Changes to `~/local_Documents/CausalInferenceWithR/Project.toml`
  No Changes to `~/local_Documents/CausalInferenceWithR/Manifest.toml`


In [126]:
UnequalVarianceTTest(y2[@. r1 == 1], y2[@. r1 == 0]) # 繰り返すが，サンプルサイズが小さいので同じ結果にはならない。

Two sample t-test (unequal variance)
------------------------------------
Population details:
    parameter of interest:   Mean difference
    value under h_0:         0
    point estimate:          12.5152
    95% confidence interval: (4.219, 20.81)

Test summary:
    outcome with 95% confidence: reject h_0
    two-sided p-value:           0.0058

Details:
    number of observations:   [11,9]
    t-statistic:              3.2178051903997393
    degrees of freedom:       14.877550058900114
    empirical standard error: 3.8893440636152343
